In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
!pip install noisereduce

import pandas as pd
import numpy as np
import random
from glob import glob
import os
import librosa
import noisereduce as nr

import warnings
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
cd "/content/drive/MyDrive/LY Project/"

/content/drive/.shortcut-targets-by-id/1ty6Q2chm7khBo3iSNsdx5NVqeOpCUmjM/LY Project


In [9]:
df = pd.read_csv('/content/drive/MyDrive/LY Project/concat-crema-demotions_data_features.csv')
print('The dataset has {} audio files'.format(df.shape[0]))
df.sample(5)

The dataset has 11970 audio files


,Unnamed: 0,path,filename,dataset,duration,gender,age,emotion,f0_mean,f0_median,...,mfcc21_var,mfcc22_var,mfcc23_var,mfcc24_var,mfcc25_var,mfcc26_var,mfcc27_var,mfcc28_var,mfcc29_var,mfcc30_var
5380,5380,cremad/AudioWAV/1066_TIE_SAD_XX.wav,1066_TIE_SAD_XX,CREMA-D,2.936,male,25,sad,108.175293,110.317882,...,11.884626,11.107758,11.503146,10.261239,14.155341,15.866578,10.888888,7.410679,16.984173,21.221403
9770,9770,toronto-emotional-speech-set-tess/TESS Toronto...,OAF_angry/OAF_back_angry,TESS,1.539,female,26,angry,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1326,1326,cremad/AudioWAV/1017_IOM_SAD_XX.wav,1017_IOM_SAD_XX,CREMA-D,2.135,male,42,sad,139.587826,107.487782,...,23.135113,35.571090,42.334663,19.199726,18.567259,12.559933,11.294953,18.991983,12.955100,10.873522
2515,2515,cremad/AudioWAV/1031_TSI_NEU_XX.wav,1031_TSI_NEU_XX,CREMA-D,2.502,male,31,neutral,NaN,NaN,...,18.404442,12.256306,17.157707,23.131132,19.411875,9.688784,13.868891,12.784361,7.250336,12.318986
3853,3853,cremad/AudioWAV/1048_IEO_NEU_XX.wav,1048_IEO_NEU_XX,CREMA-D,1.835,male,38,neutral,137.129689,137.397657,...,28.919846,19.246609,17.061031,16.323881,6.324976,26.473700,17.039118,12.383289,5.902890,10.164204


In [13]:
# df has 7442 values of crema-d with 97 features. However, other datasets features have not been extracted, hence drop
df = df.drop(df[df['dataset'] != 'CREMA-D'].index)

In [18]:
df['f0_mean'].isna().sum()

480

In [19]:
# dropping null samples
df = df[df['f0_mean'].notna()]

In [21]:
df.to_csv("cleaned-crema-features.csv")

In [28]:
df = pd.read_csv("cleaned-crema-features.csv")

In [32]:
df = df.drop(df.columns[0], axis=1)

In [34]:
df.sample()

,path,filename,dataset,duration,gender,age,emotion,f0_mean,f0_median,f0_std,...,mfcc21_var,mfcc22_var,mfcc23_var,mfcc24_var,mfcc25_var,mfcc26_var,mfcc27_var,mfcc28_var,mfcc29_var,mfcc30_var
947,cremad/AudioWAV/1013_IWL_HAP_XX.wav,1013_IWL_HAP_XX,CREMA-D,2.469,female,22,happy,233.57452,227.100886,22.055895,...,68.14234,50.404453,50.862385,84.65204,140.17407,183.19267,274.57724,266.87436,131.89716,82.9597


In [35]:
# Convert target to numerical values
df['emotion'].replace({'angry':0,'disgust':1,'fear':2,'happy':3,'neutral':4,'sad':5,'surprise':6}, inplace=True)
df['gender'].replace({'female':0,'male':1}, inplace=True)

# Create feature matrix (X) and numerical target (y)
X = df.drop(columns=['path','filename','dataset','duration','age','emotion'])
y = df.emotion.values

# Split the data
from sklearn.model_selection import train_test_split

# Create train and test set
X_tr, X_te, y_tr, y_te = train_test_split(X, y, train_size=0.9, shuffle=True, stratify=y, random_state=0)

# Checking size
print('Shape of X_tr: ', X_tr.shape)
print('Shape of y_tr: ', y_tr.shape)
print('Shape of X_te: ', X_te.shape)
print('Shape of y_te: ', y_te.shape)

Shape of X_tr:  (6265, 90)
Shape of y_tr:  (6265,)
Shape of X_te:  (697, 90)
Shape of y_te:  (697,)


In [36]:
X.sample()

,gender,f0_mean,f0_median,f0_std,f0_0,f0_25,f0_75,f0_100,zcr_mean,zcr_var,...,mfcc21_var,mfcc22_var,mfcc23_var,mfcc24_var,mfcc25_var,mfcc26_var,mfcc27_var,mfcc28_var,mfcc29_var,mfcc30_var
3220,0,208.999347,203.495938,17.466126,185.531823,197.702804,214.354693,271.634489,0.067644,0.000307,...,19.623095,20.23716,19.086687,21.818592,23.57843,56.762398,53.05542,71.03225,68.01681,54.23741


In [37]:
# Scale the data
from sklearn.preprocessing import StandardScaler

# Create Standard scaler
scaler = StandardScaler()

# Rescale data
X_tr_rescaled = scaler.fit_transform(X_tr)
X_te_rescaled = scaler.transform(X_te)

In [38]:
%%time

# importing necessary libraries for this task
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

# Create a random forest classifier
rf = RandomForestClassifier(class_weight="balanced", random_state=0)

# Create k-fold object for cross-validation with shuffle
kfold = KFold(n_splits=5, shuffle=True, random_state=0)

# Create a list of parameter
param_grid = {
    'n_estimators': [1, 5, 10, 50, 100, 200, 300],      # number of trees to search
    'max_depth': [1, 10, 25, 50, 75, 100, None]         # depths for the random forest model
}

# Instantiate the grid search model
grid_cv = GridSearchCV(estimator = rf, param_grid = param_grid, cv = kfold, n_jobs = -1, return_train_score=1)

# Fit estimator
grid_cv.fit(X_tr_rescaled, y_tr)

CPU times: user 16.8 s, sys: 577 ms, total: 17.4 s
Wall time: 12min 40s


GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=0),
             n_jobs=-1,
             param_grid={'max_depth': [1, 10, 25, 50, 75, 100, None],
                         'n_estimators': [1, 5, 10, 50, 100, 200, 300]},
             return_train_score=1)

In [39]:
# Collect results in a DataFrame
cv_results = pd.DataFrame(grid_cv.cv_results_)

# Print the columns with score and hyperparameters
cols = [
    "mean_test_score",
    "param_max_depth",
    "param_n_estimators",
]

# Display fist 5 columns with higher mean validation score
cv_results[cols].sort_values("mean_test_score", ascending=False).head()

,mean_test_score,param_max_depth,param_n_estimators
19,0.522426,25,200
26,0.522107,50,200
47,0.522107,None,200
40,0.522107,100,200
33,0.522107,75,200


In [42]:
cv_results.to_csv("crema-randomforest.csv")